In [1]:
import os
import json 
import pandas as pd
import traceback

In [5]:
from langchain.chat_models import ChatOpenAI

ModuleNotFoundError: No module named 'langchain_community'

In [7]:
from langchain.chat_models import ChatOpenAI

In [10]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [ ]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)